# Washington DC Biking data | Hourly Bike Count Prediction
MBD O-1-5

### Notebook preperation

In [14]:
# Imports
%matplotlib inline


from My_Functions import * 
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from collections import defaultdict

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score,roc_curve
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict,validation_curve
from sklearn.ensemble import RandomForestRegressor


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler,PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator , MultipleLocator
##from matplotlib import XAxis
from gplearn.genetic import SymbolicRegressor

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer,MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression,LinearRegression, OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split , TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from matplotlib.gridspec import GridSpec
import plotly.tools as tls
import plotly
plotly.tools.set_credentials_file(username='Furqan92', api_key='22DfVN5rFRg79OYygN5h')
import plotly.plotly as py
from sklearn.decomposition import PCA
from pandas import DataFrame 
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action='ignore')
random_seed = 1234

In [22]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Data Import
hourly_raw_data = pd.read_csv('hour.csv')

# Feature Engineering

### Converting `dteday` to date

In [17]:
hourly_raw_data['dteday']=pd.to_datetime(hourly_raw_data['dteday'], format='%Y-%m-%d')

## Add `isDaylight` and  `isNoon` for hourly data

In [18]:
hourly_raw_data['isDaylight']=0
hourly_raw_data['isNoon']=0

hourly_raw_data = hourly_raw_data.apply(lambda x: isDaylight(x), axis=1)

### Adding the temp atemp windspeed hum relative to the last 7 days value

In [19]:
to_relative  = ['temp', 'atemp', 'hum','windspeed']
hourly_raw_data = relative_values(hourly_raw_data, to_relative)

### Adding `RushHour-High` & 	`RushHour-Med`  &	`RushHour-Low`

In [23]:
hourly_raw_data['RushHour-High'] = 0
hourly_raw_data['RushHour-Med'] = 0
hourly_raw_data['RushHour-Low'] = 0

hourly_raw_data = hourly_raw_data.apply(lambda x: addRushHourFlags(x), axis=1)

## Splitting Data

In [30]:
wd_hr = num_name(hourly_raw_data.loc[(hourly_raw_data['workingday'].isin([1]) )])
we_hr = num_name(hourly_raw_data.loc[(~hourly_raw_data['workingday'].isin([1]) )])

## Mean of the past 3 weeks during the same hour

In [31]:
wd_hr= mean_per_hour_3weeks(wd_hr)
we_hr = mean_per_hour_3weeks(we_hr)

### One hot Encoding | 2x for splitted datasets
For `season`, `weathersit`, `mnth`,`weekday`,`hr`

In [9]:
category  = ['season', 'weathersit', 'mnth','weekday','hr']

wd_h = onehot_encode(wd_hr,category)
wd_h  = wd_h.drop('instant',axis=1)

we_h = onehot_encode(we_hr,category)
we_h  = we_h.drop('instant',axis=1)

## Genetic Programming | 2x for splitted datasets

In [10]:
dates = wd_h['dteday']
registered = wd_h['registered']
casual = wd_h['casual']
wd_h = Genetic_P(wd_h.drop(['registered','casual','dteday'],axis=1),'cnt')
wd_h['dteday'] = dates
wd_h['registered'] = registered
wd_h['casual'] = casual

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.18 0.09942326137358169        2 0.6505320758276544 0.6061161019308257      1.26m
   1     7.37 0.3405230277989632       29 0.7217874206539634 0.7108497795406704      1.26m
   2     4.66 0.5835678514401713       11 0.7495351700970283 0.7491743968591893      1.22m
   3     7.83 0.5793949786595347       15 0.7739568950635518 0.751786033765289      1.18m
   4    11.38 0.5724228903723381       15 0.7717186747501493 0.7714686861273462      1.15m
   5    17.76 0.6233817962149497       32 0.7847511039528214 0.7823815611133634      1.16m
   6    18.67 0.6245263650355682       38 0.7884781245919402 0.7845298666040835      1.14m
   7    22.53 0.6578500015008693       37 0.7926472965613747 0.7855220308284739      1.10m
   8    23.71 0.66847645305

In [11]:
dates = we_h['dteday']
registered = we_h['registered']
casual = we_h['casual']
we_h = Genetic_P(we_h.drop(['registered','casual','dteday'],axis=1),'cnt')
we_h['dteday'] = dates
we_h['registered'] = registered
we_h['casual'] = casual

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.25 0.12759481414554638        7 0.6941021639536878 0.661031258981936     25.14s
   1     7.13 0.42251296768700547        8 0.7800627241656242 0.7695711401856637     41.22s
   2     5.61 0.6223073326626798        7 0.7857945663747576 0.79758731555797     47.61s
   3     6.87 0.640091115813204        7 0.7904298460608005 0.7561691080505744     49.10s
   4      5.7 0.7028417383567165       12 0.808636521758778 0.8060500635309477     48.93s
   5     8.06 0.6947282806689465       25 0.8195412204156118 0.8363727267724078     47.81s
   6    11.11 0.7334126303541902       31 0.8494396948049476 0.8319766826691746     46.19s
   7    12.19 0.7389363580347248       26 0.8618152475368851 0.860827079724182     44.13s
   8    19.14 0.746179703704376

# Final Datasets

In [29]:
we_h[np.arange(1,15)]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
6,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333
7,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760
8,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284
9,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577
10,0.941984,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857
11,0.861323,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433,0.869433
12,1.005714,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852,1.016852
13,1.064361,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824
14,1.064361,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824,1.077824
15,1.035729,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029,1.048029


In [13]:
we_h.head()

,yr,atemp,isNoon,RushHour-High,workingday,windspeed,cnt,RushHour-Low,temp,isDaylight,...,8,9,10,11,12,13,14,dteday,registered,casual
6,0,0.2727,0,0,0,0.0000,2,1,0.22,0,...,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,0.269333,2011-01-01,0,2
7,0,0.2576,0,0,0,0.0000,3,1,0.20,0,...,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,0.254760,2011-01-01,2,1
8,0,0.2879,0,0,0,0.0000,8,0,0.24,1,...,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,0.386284,2011-01-01,7,1
9,0,0.3485,0,0,0,0.0000,14,0,0.32,1,...,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,0.464577,2011-01-01,6,8
10,0,0.3939,0,1,0,0.2537,36,0,0.38,1,...,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,0.950857,2011-01-01,24,12


## Save Both datasets

In [18]:
wd_h.to_csv("workingdays_data_prepared.csv", index=False)
we_h.to_csv("weekends_holi_data_prepared.csv", index=False)

In [32]:
wd_h.columns

Index([                'yr',              'atemp',             'isNoon',
            'RushHour-High',         'workingday',          'windspeed',
                      'cnt',       'RushHour-Low',               'temp',
               'isDaylight',            'holiday',      'relative_temp',
             'RushHour-Med',     'relative_atemp',       'relative_hum',
       'relative_windspeed',                'hum',             's_fall',
                 's_spring',           's_summer',           's_winter',
             'w_Acceptable',              'w_Bad',            'w_Chaos',
                   'w_Good',                'm_1',                'm_2',
                      'm_3',                'm_4',                'm_5',
                      'm_6',                'm_7',                'm_8',
                      'm_9',               'm_10',               'm_11',
                     'm_12',                'w_1',                'w_2',
                      'w_3',                'w_4', 